In [1]:
import h5py as h5
#from h5glance import H5Glance
import numpy as np
import gvar as gv
import pandas as pd
import os 
import sys
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib notebook
#import h5ls
import lsqfit
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
import yaml
from pathlib import Path
import platform 

In [2]:
#import local files
#sys.path.append("../")
# import data_loader as dl
# import fitter as fit
# import fit_manager as fm
import gmo_fitter as gmo 
import gmo_fit_analysis as fa
import mass_relations as ma 
import i_o 
import load_data_priors as ld

matplotlib.rcParams['figure.figsize'] = [10, 8]

In [3]:
if platform.system() == 'Darwin':
    file = '/Users/grantdb/lqcd/data/c51_2pt_octet_decuplet.h5'
else:
    file = '/home/gmoney/lqcd/data/c51_2pt_octet_decuplet.h5'

with h5.File(file,"r") as f:
    ensembles = {}
    ensembles = list(f.keys())
# ensembles

In [4]:
import importlib
# fit_params = 'input_files/a09m135.py'
# fit_params = 'input_files/a12m220.py'
# fit_params = 'input_files/a12m260.py'
fit_params = 'input_files/a12m310.py'



sys.path.append(os.path.dirname(os.path.abspath(fit_params)))
fp = importlib.import_module(
    fit_params.split('/')[-1].split('.py')[0])
p_dict = fp.p_dict

**

In [5]:
# gmo_fits = {}  # C_2pt(t) for each baryon at each time slice t
corr_fits = {} # resulting fit params, need E0 for gmo study 
mass = {}
meson_mass = {}
gmo_states = ['sigma','xi','proton','lam','piplus','kplus']
# meson_states = ['piplus','kplus']
# ld.get_raw_corr(file_h5=file,abbr='a09m135')

# for ens in ensembles:
mass[p_dict['abbr']] = {}
meson_mass[p_dict['abbr']] = {}

out_path = 'fit_results/{0}/'.format(p_dict['abbr'])
# meson_out_path = 'fit_results/{0}/'.format(p_dict['abbr'])
for corr in gmo_states:
#     gmo_fits = gv.load(out_path+'fit_data')
    corr_fits = gv.load(out_path+'fit_params_all')
    # meson_fits = gv.load(out_path +'meson_fit_params')
    mass[p_dict['abbr']][corr] = corr_fits['p'][corr+'_E0']
        
mass

{'a12m310': {'sigma': 0.7606(19),
  'xi': 0.8063(11),
  'proton': 0.6569(29),
  'lam': 0.7249(17),
  'piplus': 0.18882(15),
  'kplus': 0.32435(17)}}

In [7]:
importlib.reload(ma)
importlib.reload(i_o)
abbr =p_dict['abbr']
# for key in gmo_fits.keys():
# gmo = {}
gmo_raw =  ma.GMO(mass_dict=mass,file=file,abbr=abbr)
gmo_out = gmo_raw._G_gmo(log=False)
print(gmo_out)
mass[abbr]['eta'] = gmo_raw._get_eta_mass()
print(mass)
eff_mass = ma.gmo_eff_mass(gmo_out=gmo_out,dt=1)
# print(eff_mass)
ma.plot_effective_mass(eff_mass,t_plot_min=1,t_plot_max=25,fig_name='eff_mass')
print(gmo_raw._gmo_violation())
# get fpi data, TODO get bs mpi,mk,m_baryon data from this concatenated h5 file
input_output = i_o.InputOutput()
data= input_output.get_data(scheme='w0_org',units='phys',include_phys=False)
#ensembles.clear
#ensembles.insert['a09m135','a09m220']
phys_point_data = input_output.get_data_phys_point()
#data.get_data()
# data,ensembles

{'SS': array([0.60648(67), 0.605407(27), 0.604891(51), 0.604357(70),
       0.603787(92), 0.60320(13), 0.60256(18), 0.60181(25), 0.60090(36),
       0.59972(55), 0.59847(84), 0.5971(13), 0.5959(19), 0.5945(29),
       0.5910(44), 0.5845(63), 0.5759(90), 0.569(13), 0.560(18),
       0.555(26), 0.562(39), 0.584(63), 0.583(84), 0.558(92), 0.57(12),
       0.61(18), 0.66(29), 0.69(41), 0.76(79), 2(26), nan +- nan,
       0.23(47), 0.24(17), 0.31(13), 0.38(12), 0.50(14), 0.65(27),
       1.2(2.2), nan +- nan, nan +- nan, 0.63(86), -0.1(1.8), -1.1(4.2),
       nan +- nan, nan +- nan, nan +- nan, nan +- nan, nan +- nan,
       nan +- nan, nan +- nan, nan +- nan, nan +- nan, nan +- nan,
       1.9(7.6), 0.81(34), 0.689(89), 0.640(31), 0.630(13), 0.6217(59),
       0.6128(27), 0.6073(13), 0.60483(64), 0.60460(27), 0.605354(52)],
      dtype=object), 'PS': array([0.6102(13), 0.605489(30), 0.604934(61), 0.604275(80), 0.60358(10),
       0.60291(14), 0.60221(19), 0.60138(26), 0.60044(37), 0.59946(

/home/gmoney/lqcd/gmo_relation/mass_relations.py:196: RuntimeWarning: invalid value encountered in power
  * np.power(temp[('sigma_p', smr)], 1/3)
/home/gmoney/lqcd/gmo_relation/mass_relations.py:197: RuntimeWarning: invalid value encountered in power
  * np.power(temp[('proton', smr)], -2/3)
/home/gmoney/lqcd/gmo_relation/mass_relations.py:198: RuntimeWarning: invalid value encountered in power
  * np.power(temp[('xi_z', smr)], -2/3)


<IPython.core.display.Javascript object>

('gmo rln:', 0.0030(27), 'single octet:', 0.7416(11), 'gmo violation as ratio of above:', 0.0041(37))


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/gmoney/lqcd/data/input_data.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
data[abbr]['Fpi']

99.78(74)

In [ ]:
importlib.reload(ma)
gmo_xpt = ma.GMO_xpt(mass_dict=mass,abbr=abbr,data=data,loop_level=True,delta=293)
gmo_xpt._lo_gmo_ratio_exp()

0.003654(64)

In [ ]:
mpi_fpi_ratio = np.power(260,2) / np.power(data['Fpi'],2) 
mpi_fpi_ratio

KeyError: 'Fpi'

In [ ]:
gmo_xpt._lo_gmo()

0.00002781(41)

In [ ]:
eff_mass = ma.gmo_eff_mass(gmo_out=gmo_out,dt=1)
# eff_mass

In [ ]:
ma.plot_log_gmo(gmo_out ,0, 7,fig_name='gmo_test')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
ma.plot_effective_mass(eff_mass,t_plot_min=1,t_plot_max=8,fig_name='eff_mass')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>